# Tensorflow: IBACopy

This notebook shows how to use the [IBACopy]() class. It is very flexible and should work with different models.
For classification models, you might want to look at [IBACopyInnvestigate]().
If you can add additional layers to your model, it might also be worth looking at [IBALayer]().

Ensure that `./imagenet` points to your copy of the ImageNet dataset. 

You might want to create a symlink:

In [ ]:
# ! ln -s /path/to/your/imagenet/folder/ imagenet 

In [ ]:
# select your device
# %env CUDA_VISIBLE_DEVICES=0


# reduce tensorflow noise
import warnings
warnings.filterwarnings("ignore")

import sys
import os

from tqdm.notebook import tqdm
    
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

import tensorflow.compat.v1 as tf

import keras
import keras.backend as K
from keras.applications.resnet50 import preprocess_input, ResNet50
from keras.applications import VGG16 

from skimage.transform import resize

from IBA.utils import plot_saliency_map
from IBA.tensorflow_v1 import IBACopy, TFWelfordEstimator, model_wo_softmax, to_saliency_map 


In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
keras.backend.set_session(sess)

In [ ]:
print("TensorFlow version: {}, Keras version: {}".format(
    tf.version.VERSION, keras.__version__))

In [ ]:
# data loading

def get_imagenet_generator(val_dir, image_size = (224, 224), shuffle=True, batch_size=50):
    image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocess_input
    )
    return image_generator.flow_from_directory(
        val_dir, shuffle=shuffle, seed=0, batch_size=batch_size, target_size=image_size)

def norm_image(x):
    return (x - x.min()) / (x.max() - x.min())

imagenet_dir = "./imagenet"
imagenet_val_dir = os.path.join(imagenet_dir, "validation")

img_batch, target_batch = next(get_imagenet_generator(imagenet_val_dir))

monkey_pil = Image.open("monkeys.jpg").resize((224, 224))
monkey = preprocess_input(np.array(monkey_pil))[None]
monkey_target = 382  # 382: squirrel monkey

In [ ]:
# load model
model_softmax = VGG16(weights='imagenet')

# remove the final softmax layer
model = model_wo_softmax(model_softmax)

# select layer after which the bottleneck will be inserted 
feat_layer = model.get_layer(name='block3_conv2')

Create the Analyzer

In [ ]:
iba = IBACopy(feat_layer.output, model.output)

Double check if model was copied correctly

In [ ]:
# checks if all variables are equal in the original graph and the copied graph
iba.assert_variables_equal()

iba_logits = iba.predict({model.input: img_batch})
model_logits = model.predict(img_batch)
assert (np.abs(iba_logits - model_logits).mean()) < 1e-5
print(np.abs(iba_logits - model_logits).mean())

Fit mean and std of the feature map

In [ ]:
feed_dict_gen = map(lambda x: {model.input: x[0]}, 
                    get_imagenet_generator(imagenet_val_dir))

iba.fit_generator(feed_dict_gen, n_samples=5000)
print("Fitted estimator on {} samples".format(iba._estimator.n_samples()))

In [ ]:
# set classification loss
target = iba.set_classification_loss()

# you can also specificy your own loss with :
# iba.set_model_loss(my_loss)

In [ ]:
# get the saliency map 
capacity = iba.analyze(
    feature_feed_dict={model.input: monkey},
    copy_feed_dict={target: np.array([monkey_target])}    
)
saliency_map = to_saliency_map(capacity, shape=(224, 224))

In [ ]:
saliency_map.shape

In [ ]:
K.image_data_format()

In [ ]:
plot_saliency_map(saliency_map, img=norm_image(monkey[0]))

## Access to internal values 

You can access all intermediate values of the optimzation through the `iba.get_report()` method.
To store the intermediate values, you have to call either `iba.collect_all()` or `iba.collect(*var_names)` before running `iba.analyze(..)`.

In [ ]:
# collect all intermediate tensors
iba.collect_all()

# storing all tensors can slow down the optimization. 
# you can also select to store only specific ones:
# iba.collect("alpha", "model_loss")
# to only collect a subset all all tensors


# run the optimization
capacity = iba.analyze(
    feature_feed_dict={model.input: monkey},
    copy_feed_dict={iba.target: np.array([monkey_target])}    
)

# get all saved outputs
report = iba.get_report()

`report` is an `OrderedDict`  which maps each `iteration` to a dictionray of `{var_name, var_value}`.
The `init` iteration is computed without an optimizer update. Values not changing such as the feature values are only included in the `init` iteration.
The `final` iteration is again computed without an optimizer update.

In [ ]:
print("iterations:", list(report.keys()))


Print all available tensors in the `init` iteration:

In [ ]:
print("{:<30} {:}".format("name:", "shape"))
print()
for name, val in report['init'].items():
    print("{:<30} {:}".format(name + ":", str(val.shape)))

### Losses during optimization

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 3))
ax[0].set_title("cross entrop loss")
ax[0].plot(list(report.keys()), [it['model_loss'] for it in report.values()])

ax[1].set_title("mean capacity")
ax[1].plot(list(report.keys()), [it['capacity_mean'] for it in report.values()])


### Distribution of alpha (pre-softmax) values per iteraton

In [ ]:
cols = 6
rows = len(report) // cols

fig, axes = plt.subplots(rows, cols, figsize=(2.8*cols, 2.2*rows))

for ax, (it, values) in zip(axes.flatten(), report.items()):
    ax.hist(values['alpha'].flatten(), log=True, bins=20)
    ax.set_title("iteration: " + str(it))
    
plt.subplots_adjust(wspace=0.3, hspace=0.5)

fig.suptitle("distribution of alpha (pre-softmax) values per iteraton.", y=1)
plt.show()

### Distributiuon of the final capacity

In [ ]:
plt.hist(report['final']['capacity'].flatten(), bins=20, log=True)
plt.title("Distributiuon of the final capacity")
plt.show()